In [1]:
import numpy as np

# Input: expects 3xN matrix of points
# Returns R,t
# R = 3x3 rotation matrix
# t = 3x1 column vector

def rigid_transform_3D(A, B):
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("det(R) < R, reflection detected!, correcting for it ...")
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [2]:
import pandas as pd

In [12]:
gcp = pd.read_csv("control_pts_update.txt", sep=' ', index_col=0)
gcp

,x,y,z,r,source
Sphere,,,,,
1,1120.500214,33.944318,-428.035216,0.326115,parkinglot_004
2,1110.760515,18.975729,-427.847971,0.326115,parkinglot_003
3,1095.679312,44.756847,-427.769038,0.326115,parkinglot_003
4,1056.491475,15.646491,-428.374438,0.326115,parkinglot_006
5,1041.841133,24.145055,-428.810535,0.326115,parkinglot_006
6,1028.593841,23.245944,-428.322305,0.326115,parkinglot_005
10,844.233104,-490.266961,-426.012824,0.326115,parkinglot_017
11,848.232238,-500.427983,-426.139751,0.326115,parkinglot_017
12,857.948074,-493.671166,-426.364404,0.326115,parkinglot_017


In [13]:
# sphere radius: 0.326115 ft
# pole height: 4.3229166667 ft

# adjust control points to ground
gcp.loc[:, 'z'] -= (0.326115 + 4.3229166667)
gcp_scanner = gcp.copy()
gcp_scanner

,x,y,z,r,source
Sphere,,,,,
1,1120.500214,33.944318,-432.684248,0.326115,parkinglot_004
2,1110.760515,18.975729,-432.497003,0.326115,parkinglot_003
3,1095.679312,44.756847,-432.418070,0.326115,parkinglot_003
4,1056.491475,15.646491,-433.023470,0.326115,parkinglot_006
5,1041.841133,24.145055,-433.459567,0.326115,parkinglot_006
6,1028.593841,23.245944,-432.971337,0.326115,parkinglot_005
10,844.233104,-490.266961,-430.661856,0.326115,parkinglot_017
11,848.232238,-500.427983,-430.788783,0.326115,parkinglot_017
12,857.948074,-493.671166,-431.013436,0.326115,parkinglot_017


In [16]:
gcp_global = pd.read_csv("Hoboken_control_with_accuracy.csv", index_col=0)
gcp_global

,lon,lat,orthoheight,h_accuracy,v_accuracy
OBJECTID,,,,,
1,623205.6986,693359.2726,4.593400,0.005000,0.007
2,623198.4926,693342.7713,4.380135,0.007810,0.013
3,623179.3188,693365.8054,4.534342,0.007211,0.011
4,623145.5895,693330.6326,4.045473,0.007211,0.011
5,623129.7283,693336.5529,3.851894,0.005831,0.009
6,623116.7280,693333.5158,3.970010,0.007211,0.010
7,623077.3298,693339.8872,3.983134,0.008062,0.011
8,623060.5526,693341.7779,4.081564,0.007810,0.011
9,623048.7078,693353.1365,3.999539,0.007211,0.010


# Matrix construction

In [19]:
# 3D rigid transformation using points [1,2,3,4,5,6]

A = np.array(gcp_scanner.loc[1:6, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[1:6, ['lon', 'lat', 'orthoheight']]).T

# Rigid transformation 3D

In [24]:
[ret_R, ret_t] = rigid_transform_3D(A, B)

# Root mean squared error

In [27]:
# Compare the recovered R and t with the original
B2 = (ret_R@A) + ret_t

n = A.shape[1]

# Find the root mean squared error
err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.17805315585945114

# Test for control point 10, 11, 12

In [28]:
# 3D rigid transformation using points [10, 11, 12]

A = np.array(gcp_scanner.loc[10:12, ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[10:12, ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

B2 = (ret_R@A) + ret_t

n = A.shape[1]

err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

det(R) < R, reflection detected!, correcting for it ...


0.02081924952889955

# Test for control point [1,2,3,4,5,6,10,11,12]

In [29]:
# 3D rigid transformation using points [1,2,3,4,5,6,10,11,12]

A = np.array(gcp_scanner.loc[[1,2,3,4,5,6,10,11,12], ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[[1,2,3,4,5,6,10,11,12], ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

B2 = (ret_R@A) + ret_t

n = A.shape[1]

err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

0.74268046306231

In [30]:
# Test for control point [1,10,18]

A = np.array(gcp_scanner.loc[[1,10,18], ['x', 'y', 'z']]).T
B = np.array(gcp_global.loc[[1,10,18], ['lon', 'lat', 'orthoheight']]).T

[ret_R, ret_t] = rigid_transform_3D(A, B)

B2 = (ret_R@A) + ret_t

n = A.shape[1]

err = B2 - B
err = err * err
err = np.sum(err)
rmse = np.sqrt(err/n)

rmse

det(R) < R, reflection detected!, correcting for it ...


3.2136878206843686